In [42]:
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

In [2]:
%run scripts/helper.py

In [3]:
crowd_train = load_file('./data/train.csv/train.csv', None)

In [4]:
# General text related features
# 1. Text length of the title

In [5]:
crowd_train.columns

Index([u'id', u'query', u'product_title', u'product_description', u'median_relevance', u'relevance_variance'], dtype='object')

In [56]:
title_length = crowd_train[crowd_train.relevance_variance < 0.5].apply(lambda x: len(x['product_title']), axis=1)

In [57]:
# lets see how correlated this feature is with response variable
# crowd_train[['title_length', 'median_relevance']].corr()
pd.DataFrame({'title_length': title_length, 
              'median_relevance': crowd_train[crowd_train.relevance_variance < 0.5].median_relevance}).corr()

,median_relevance,title_length
median_relevance,1.000000,0.042657
title_length,0.042657,1.000000


In [9]:
# Not a high correlation

In [58]:
# 2. Number of words in the title
# crowd_train['num_words_title'] = crowd_train.apply(lambda x: len(x['product_title'].split(' ')), axis=1)
num_words_title = crowd_train[crowd_train.relevance_variance < 0.5].apply(lambda x: len(x['product_title'].split(' ')), axis=1)

In [59]:
# crowd_train[['num_words_title', 'median_relevance']].corr()
pd.DataFrame({'num_words_title': num_words_title, 
              'median_relevance': crowd_train[crowd_train.relevance_variance < 0.5].median_relevance}).corr()

,median_relevance,num_words_title
median_relevance,1.00000,0.02938
num_words_title,0.02938,1.00000


In [12]:
# 3. Number of words in the prodcut description
crowd_train['num_words_desc'] = crowd_train.apply(lambda x: len(x['product_description']), axis=1)

In [13]:
crowd_train[['num_words_desc', 'median_relevance']].corr()

,num_words_desc,median_relevance
num_words_desc,1.000000,-0.009985
median_relevance,-0.009985,1.000000


In [31]:
# 4. Ratio of words in query other than stopwords that match in the title and description
def f(x):
    query = x['query'].lower()
    title = x['product_title'].lower()
    desc = x['product_description'].lower()
    stop = stopwords.words('english')
    total_words = len(title.split(' ')) + len(desc.split(' '))
    count = 0
    
    unique_query_terms = list(set(query.split(' ')))
    for q in unique_query_terms:
        if q not in stop:
            if q in title or q in desc:
                count += 1
    
    return (count * 1.) / total_words

crowd_train['ratio_query_terms_in_res'] = crowd_train.apply(f, axis=1)

In [32]:
crowd_train[['ratio_query_terms_in_res', 'median_relevance']].corr()

,ratio_query_terms_in_res,median_relevance
ratio_query_terms_in_res,1.000000,0.158718
median_relevance,0.158718,1.000000


In [5]:
# Jaccard distance between query and (title + description)
def jaccard(x):
    query = x['query'].lower()
    title = x['product_title'].lower()
    description = x['product_description'].lower()
    response = title + ' ' + description
    
    query_set = set(query.split(' '))
    response_set = set(response.split(' '))
    
    query_response_intersection_len = len(query_set & response_set)
    query_response_union_len = len(query_set | response_set)
    
    return (query_response_intersection_len * 1.) / (query_response_union_len)

crowd_train['jaccard_dist'] = crowd_train.apply(jaccard, axis=1)

In [6]:
# lets how much this variable is correlated with distance
crowd_train[['jaccard_dist', 'median_relevance']].corr()

,jaccard_dist,median_relevance
jaccard_dist,1.000000,0.181246
median_relevance,0.181246,1.000000


In [7]:
crowd_train.jaccard_dist.head()

0    0.000000
1    0.025641
2    0.166667
3    0.020833
4    0.051282
Name: jaccard_dist, dtype: float64

In [60]:
# Check if query term in response
def is_query_in_response(train):
    query_terms = train['query'].split(' ')
    response = train['product_title'] + ' ' + train['product_description']
    
    stemmer = PorterStemmer()
    query_terms_stemmed = [stemmer.stem(q) for q in query_terms]
    response_stemmed = ''.join([stemmer.stem(r) for r in response])
    stop = stopwords.words('english')
       
    keyword = False
    
    for q in query_terms_stemmed:
        if q not in stop:
            keyword = True
            if response_stemmed.lower().find(q) == -1:
                return 0
    
    if keyword == False:
        return 0
    else:
        return 1

# crowd_train['query_in_response'] = crowd_train.apply(is_query_in_response, axis=1)
query_in_response = crowd_train[crowd_train.relevance_variance < 0.5].apply(is_query_in_response, axis=1)

In [61]:
# crowd_train[['query_in_response', 'median_relevance']].corr()
pd.DataFrame({'query_in_response': query_in_response, 
              'median_relevance': crowd_train[crowd_train.relevance_variance < 0.5].median_relevance}).corr()

,median_relevance,query_in_response
median_relevance,1.000000,0.408791
query_in_response,0.408791,1.000000


In [62]:
# lets find out how many query terms found in response
def count_query_terms_in_response(train):
    query_terms = train['query'].split(' ')
    unique_terms = list(set(query_terms))
    response = train['product_title'].lower() + ' ' + train['product_description'].lower()
    stemmer = PorterStemmer()
    query_terms_stemmed = [stemmer.stem(q) for q in unique_terms]
    response_stemmed = ''.join([stemmer.stem(r) for r in response])
    stop = stopwords.words('english')
    
    
    count = 0
    for q in query_terms_stemmed:
        if q not in stop:
            if response_stemmed.find(q) != -1:
                count += 1
    return count

# crowd_train['num_terms_in_resp'] = crowd_train.apply(count_query_terms_in_response, axis=1)
num_terms_in_resp = crowd_train[crowd_train.relevance_variance < 0.5].apply(count_query_terms_in_response, axis=1)

In [63]:
# crowd_train[['num_terms_in_resp', 'median_relevance']].corr()
pd.DataFrame({'num_terms_in_resp': num_terms_in_resp, 
              'median_relevance': crowd_train[crowd_train.relevance_variance < 0.5].median_relevance}).corr()

,median_relevance,num_terms_in_resp
median_relevance,1.000000,0.317533
num_terms_in_resp,0.317533,1.000000


In [61]:
def lch_similarity(x):
    query = x['query'].lower()
    response = x['product_title'].lower() + ' ' + x['product_description'].lower()
    stop = stopwords.words('english')
    total = 0
    
    for q in query.split(' '):
        if q not in stop:
            query_noun = wn.synsets(q, pos=wn.NOUN)
            if len(query_noun) > 0:
                for r in response.split(' '):
                    if r not in stop:
                        synonyms = wn.synsets(r, pos=wn.NOUN)
                        if len(synonyms) > 0:
                            total += query_noun[0].lch_similarity(synonyms[0])
    
    return total

crowd_train['lch_similarity'] = crowd_train.apply(lch_similarity, axis=1)

In [63]:
crowd_train[['lch_similarity', 'median_relevance']].corr()

,lch_similarity,median_relevance
lch_similarity,1.000000,0.004542
median_relevance,0.004542,1.000000


### Query Length

In [6]:
def query_length(x):
    return len(x['query'].split(' '))

crowd_train['query_length'] = crowd_train.apply(query_length, axis=1)

In [7]:
crowd_train[['query_length', 'median_relevance']].corr()

,query_length,median_relevance
query_length,1.000000,-0.100067
median_relevance,-0.100067,1.000000


### Response length

In [8]:
def response_length(x):
    return len(x['product_title']) + len(x['product_description'])

crowd_train['response_length'] = crowd_train.apply(response_length, axis=1)

In [9]:
crowd_train[['response_length', 'median_relevance']].corr()

,response_length,median_relevance
response_length,1.000000,-0.008905
median_relevance,-0.008905,1.000000


In [47]:
def query_synonymns_check(x):
    query = x['query'].lower()
    query_terms = list(set(query.split()))
    response = x['product_title'].lower() + ' ' + x['product_description'].lower()
    query_synonymns = []
    stop = stopwords.words('english')
    
    for q in query_terms:
        for i, j in enumerate(wn.synsets(q)):
            query_synonymns.extend(j.lemma_names)
    
    keyword = False
    for qsynonym in query_synonymns:
        if qsynonym not in stop:
            keyword = True
            if response.find(qsynonym) == -1:
                return 0
    
    if keyword == False:
        return 0
    else:
        return 1
    
crowd_train['query_synonyms_match_count'] = crowd_train.apply(query_synonymns_check, axis=1)

In [48]:
crowd_train[['query_synonyms_match_count', 'median_relevance']].corr()

,query_synonyms_match_count,median_relevance
query_synonyms_match_count,1.000000,0.017669
median_relevance,0.017669,1.000000


In [49]:
c = crowd_train.groupby('median_relevance')

In [52]:
c.get_group(4).head().relevance_variance.describe()

count    5.000000
mean     0.188400
std      0.257977
min      0.000000
25%      0.000000
50%      0.000000
75%      0.471000
max      0.471000
Name: relevance_variance, dtype: float64

In [53]:
c.get_group(3).head().relevance_variance.describe()

count    5.000000
mean     0.571200
std      0.427608
min      0.000000
25%      0.471000
50%      0.471000
75%      0.748000
max      1.166000
Name: relevance_variance, dtype: float64

In [54]:
c.get_group(2).head().relevance_variance.describe()

count    5.000000
mean     0.722800
std      0.279154
min      0.471000
25%      0.471000
50%      0.632000
75%      1.020000
max      1.020000
Name: relevance_variance, dtype: float64

In [55]:
c.get_group(1).head().relevance_variance.describe()

count    5.000000
mean     0.094200
std      0.210638
min      0.000000
25%      0.000000
50%      0.000000
75%      0.000000
max      0.471000
Name: relevance_variance, dtype: float64